In [4]:
import torch
print(torch.cuda.is_available())      
print(torch.cuda.device_count())  

True
2


### Load dataset

Dataset from: https://captain-whu.github.io/DOTA/dataset.html
Full list of albumations functions in ultralytics: https://docs.ultralytics.com/reference/data/augment/h=albumentation#ultralyticsdataaugmentBaseTransform

`convert_dota_to_yolo_obb` <br>
Converts DOTA dataset annotations to YOLO OBB (Oriented Bounding Box) format. The function processes images in the 'train' and 'val' folders of the DOTA dataset. For each image, it reads the associated label from the original labels directory and writes new labels in YOLO OBB format to a new directory.

`split_trainval` <br>
Restructures dataset directory and applies image augmentation to enrich dataset. 
```
data_root/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/
```

And the output directory structure is:

```
save_dir/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/
```

In [ ]:
from ultralytics.data.converter import convert_dota_to_yolo_obb
from ultralytics.data.split_dota import split_test, split_trainval

convert_dota_to_yolo_obb("DOTAv1.5")

# split train and val set, with labels.
split_trainval(
    data_root="DOTAv1.5/",
    save_dir="DOTAv1.5_processed/",
    rates=[0.5, 1.0, 1.5],  # Generate different versions or crops of the images
    gap=500, # Split into patches with a specific overlap or gap
)
# # split test set, without labels.
# split_test(
#     data_root="DOTAv1.0/",
#     save_dir="DOTAv1.0-split/",
#     rates=[0.5, 1.0, 1.5],  # multiscale
#     gap=500,
# )

### Inspect dataset

In [5]:
from pathlib import Path

root = Path("DOTAv1.5")
image_exts = {".jpg", ".jpeg", ".png"}
label_exts = {".txt"}

# Count images in each subdirectory under DOTAv1.5/images
print("DOTAv1.5")
print("Image counts:")
for subdir in (root / "images").iterdir():
    if subdir.is_dir():
        count = sum(1 for file in subdir.rglob("*") if file.suffix.lower() in image_exts)
        print(f"  {subdir.name}: {count}")

# Count labels in each subdirectory under DOTAv1.5/labels
print("\nLabel counts:")
for subdir in (root / "labels").iterdir():
    if subdir.is_dir():
        count = sum(1 for file in subdir.rglob("*") if file.suffix.lower() in label_exts)
        print(f"  {subdir.name}: {count}")
        if subdir.name == "val_original":
            print()  # Extra line break after 'val_original'

# Repeat for DOTAv1.5_processed
root2 = Path("DOTAv1.5_processed")
print("\nDOTAv1.5_processed")
print("Image counts:")
for subdir in (root2 / "images").iterdir():
    if subdir.is_dir():
        count = sum(1 for file in subdir.rglob("*") if file.suffix.lower() in image_exts)
        print(f"  {subdir.name}: {count}")

print("\nLabel counts:")
for subdir in (root2 / "labels").iterdir():
    if subdir.is_dir():
        count = sum(1 for file in subdir.rglob("*") if file.suffix.lower() in label_exts)
        print(f"  {subdir.name}: {count}")
        if subdir.name == "val_original":
            print()  # Extra line break after 'val_original'

DOTAv1.5
Image counts:
  train: 1411
  val: 395

Label counts:
  train: 1411
  val: 395
  train_original: 1411
  val_original: 458


DOTAv1.5_processed
Image counts:
  train: 103421
  val: 0

Label counts:
  train: 58187
  val: 0


### Load model

In [6]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO("models/yolo11n-obb.pt") # n for nano, x for huge

### Train model
Includes list of augmentations to apply to each batch before training loop. Doesn't create more images, just augments the existing ones (and labels accordingly) to enrich dataset. 

In [8]:
results = model.train(
    data="dotav1.yml",  # Path to custom dataset YAML
    epochs=100,                          # Total number of training epochs
    imgsz=640,                           # Input image size for training
    patience=15,                         # patience: Number of epochs with no improvement before early stopping
    batch=16,                            # batch: Batch size (use -1 or a fraction for auto GPU memory utilization)
    save=True,                           # save: Save training checkpoints and final model weights
    save_period=-1,                      # save_period: Frequency (in epochs) for saving checkpoints; -1 disables
    cache=False,                         # cache: Enable caching of dataset images (True for RAM, 'disk' for disk caching)
    device='0, 1',                       # device: Specify the computational device (e.g., 0, "cpu", or list of GPUs)
    workers=8,                           # workers: Number of worker threads for data loading
    project="test_yolo",                 # project: Name of the project directory for saving outputs
    name="experiment_1",                 # name: Name of the training run
    exist_ok=False,                      # exist_ok: Allow overwriting an existing project/name directory if True
    pretrained=True,                     # pretrained: Start from a pretrained model or provide a path to one
    optimizer="auto",                    # optimizer: Choice of optimizer (e.g., 'SGD', 'Adam', or 'auto')
    seed=18,                             # seed: Random seed for reproducibility
    single_cls=False,                    # single_cls: Treat all classes as a single class (useful for binary tasks)
    classes=None,                        # classes: List of class IDs to train on; None uses all classes
    rect=False,                          # rect: Enable rectangular training to minimize padding in batches
    multi_scale=True,                    # multi_scale: Enable multi-scale training by varying the imgsz during training
    cos_lr=False,                        # cos_lr: Use a cosine learning rate scheduler
    close_mosaic=10,                     # close_mosaic: Disable mosaic augmentation in the last N epochs
    resume=False,                        # resume: Resume training from the last checkpoint if available
    amp=True,                            # amp: Enable Automatic Mixed Precision training
    #fraction=1.0,                       # fraction: Fraction of the dataset to use for training
    freeze=None,                         # freeze: Freeze the first N layers or specify list of layers to freeze
    lr0=0.01,                            # lr0: Initial learning rate
    lrf=0.001,                           # lrf: Final learning rate as a fraction of lr0 (lr0 * lrf)
    momentum=0.937,                      # momentum: Momentum factor for the optimizer
    weight_decay=0.0005,                 # weight_decay: L2 regularization coefficient to prevent overfitting
    warmup_epochs=3.0,                   # warmup_epochs: Number of epochs for learning rate warmup
    warmup_momentum=0.8,                 # warmup_momentum: Initial momentum value during the warmup phase
    warmup_bias_lr=0.1,                  # warmup_bias_lr: Learning rate for bias parameters during warmup
    box=7.5,                             # box: Weight of the box loss component (bounding box regression)
    cls=0.5,                             # cls: Weight of the classification loss component
    dfl=1.5,                             # dfl: Weight of the distribution focal loss for fine-grained classification
    pose=12.0,                           # pose: Weight of the pose loss for keypoint or pose estimation tasks
    kobj=2.0,                            # kobj: Weight of the keypoint objectness loss in pose estimation models
    nbs=64,                              # nbs: Nominal batch size for loss normalization
    overlap_mask=True,                   # overlap_mask: Merge overlapping masks into a single mask if True
    mask_ratio=4,                        # mask_ratio: Downsample ratio for segmentation masks
    dropout=0.0,                         # dropout: Dropout rate for regularization
    val=True,                            # val: Enable validation during training
    plots=True,                          # plots: Generate and save plots of training and validation metrics
    hsv_h=0.015,  # Adjusts the hue of the image by a fraction of the color wheel
    hsv_s=0.7,    # Alters the saturation for varied color intensity
    hsv_v=0.4,    # Modifies brightness to simulate different lighting conditions
    degrees=0.0,  # Rotates the image 0° (no rotation); adjust between 0 and 180 for random rotation
    translate=0.1,  # Translates the image by 10% of its size
    scale=0.5,    # Scales the image, simulating objects at different distances
    shear=0.0,    # Shears the image 0° (no shearing); modify between -180 and 180 for effect
    perspective=0.0,  # Applies no perspective transformation; use values up to 0.001 if needed
    flipud=0.0,   # Probability (0%) of flipping the image upside down
    fliplr=0.5,   # 50% chance to horizontally flip the image
    bgr=0.0,      # No conversion from RGB to BGR (0% chance)
    mosaic=1.0,   # Enables mosaic augmentation by merging four images into one
    mixup=0.0,    # Disables mixup augmentation (blends two images)
    cutmix=0.0,   # Disables cutmix augmentation (combines portions of two images)
    copy_paste=0.0,  # Disables copy-paste augmentation for segmentation
    copy_paste_mode="flip",  # Use 'flip' strategy if copy-paste is enabled
    auto_augment="randaugment",  # Applies RandAugment, a random augmentation policy for classification
    erasing=0.4    # Randomly erases regions (40% probability) to encourage robust feature extraction
)

New https://pypi.org/project/ultralytics/8.3.122 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.121 🚀 Python-3.10.17 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
                                                        CUDA:1 (NVIDIA RTX 6000 Ada Generation, 48647MiB)
engine/trainer: task=obb, mode=train, model=models/yolo11n-obb.pt, data=dotav1.yml, epochs=100, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, 1, workers=8, project=test_yolo, name=experiment_14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=18, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visual

  6%|▌         | 110M/1.86G [00:00<00:09, 196MB/s] 

100%|██████████| 1.86G/1.86G [00:11<00:00, 177MB/s]
Unzipping /home/jupyter-dai7591/yolo/datasets/DOTAv1.5.zip to /home/jupyter-dai7591/yolo/datasets/DOTAv1.5...: 100%|██████████| 6554/6554 [00:08<00:00, 797.96file/s]  

Dataset download success ✅ (20.0s), saved to /home/jupyter-dai7591/yolo/datasets



Overriding model.yaml nc=80 with nc=16

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

...

                   all        458      69565      0.749      0.385      0.436      0.315

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     99/100      22.8G      1.039     0.7075      1.091          3        352: 100%|██████████| 89/89 [00:06<00:00, 14.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [00:02<00:00, 22.58it/s]


                   all        458      69565      0.743      0.391      0.437      0.316
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 84, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

99 epochs completed in 0.271 hours.
Optimizer stripped from test_yolo/experiment_14/weights/last.pt, 5.6MB
Optimizer stripped from test_yolo/experiment_14/weights/best.pt, 5.6MB

Validating test_yolo/experiment_14/weights/best.pt...
Ultralytics 8.3.121 🚀 Python-3.10.17 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
                                                        CUDA:1 (NVIDIA RTX 6000 Ada Generation, 48647MiB)
YOLO11n-obb summary (fused): 109 layers, 2,656,843 parameters, 0 gradients, 6.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [00:07<00:00,  8.05it/s]


                   all        458      69565       0.75       0.39      0.442      0.319
                 plane         71       2550      0.838      0.673      0.732      0.554
                  ship        138      10765      0.796      0.441      0.528      0.332
          storage tank         60       2940       0.87      0.223      0.343      0.226
      baseball diamond         53        213      0.781      0.446       0.55      0.384
          tennis court         93        763      0.879      0.898      0.925       0.87
      basketball court         39        143      0.696      0.378      0.444      0.391
    ground track field         71        145      0.584      0.421      0.422      0.341
                harbor        116       2102      0.734      0.695      0.705      0.384
                bridge         75        466      0.578      0.103      0.139      0.055
         large vehicle        158       5139      0.791      0.693       0.73      0.547
         small vehicl

### Validated Model

In [12]:
# Validate the model
model = YOLO("test_yolo/experiment_14/weights/best.pt") # n for nano, x for huge
metrics = model.val(data="dotav1.yml")  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category



Ultralytics 8.3.121 🚀 Python-3.10.17 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
YOLO11n-obb summary (fused): 109 layers, 2,656,843 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3591.6±1738.3 MB/s, size: 16405.2 KB)


val: Scanning /home/jupyter-dai7591/yolo/DOTAv1.5/labels/val... 3 images, 0 backgrounds, 0 corrupt:   1%|          | 3/395 [00:00<00:16, 23.28it/s]

val: Scanning /home/jupyter-dai7591/yolo/DOTAv1.5/labels/val... 395 images, 0 backgrounds, 0 corrupt: 100%|██████████| 395/395 [00:07<00:00, 50.16it/s] 

val: New cache created: /home/jupyter-dai7591/yolo/DOTAv1.5/labels/val.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:11<00:00,  2.15it/s]


                   all        395      51217      0.695      0.427      0.459      0.243
                 plane         67       2432      0.833       0.67      0.729      0.474
                  ship        126      10015      0.784      0.444      0.512      0.167
          storage tank         44        711      0.904      0.279      0.404      0.281
      baseball diamond         50        209      0.807      0.459      0.557      0.325
          tennis court         80        656       0.91      0.912       0.95      0.567
      basketball court         34        122      0.721      0.443      0.484      0.273
    ground track field         48         74      0.567      0.595      0.601      0.322
                harbor        109       2032      0.606      0.606      0.519      0.165
                bridge         58        386      0.372     0.0675     0.0777     0.0203
         large vehicle        133       4271      0.565      0.529      0.409      0.147
         small vehicl

array([    0.47361,     0.16705,     0.28117,      0.3246,     0.56738,     0.27285,      0.3218,     0.16539,    0.020274,     0.14662,     0.10721,      0.2391,    0.087511,     0.23862,     0.23657,     0.24332])